In [55]:
from ml_aos.dataloader import Donuts
import pandas as pd
import numpy as np

In [13]:
# data loader for the test set
donuts = Donuts("test", transform=False)
print(len(donuts))

27161


In [120]:
# loop through the test set and extract metadata, images, and zernikes
meta = []
images = []
zernikes = []
dofs = []
for i in range(len(donuts)):
    # load the donut data
    data = donuts[i]

    # determine which corner we're in
    fx, fy = data["field_x"].item(), data["field_y"].item()
    if fx < 0 and fy < 0:
        corner = "R00"
    elif fx < 0 and fy > 0:
        corner = "R40"
    elif fx > 0 and fy > 0:
        corner = "R44"
    else:
        corner = "R04"

    meta.append([
        data["pntId"], 
        data["obsId"], 
        data["objId"],
        fx,
        fy,
        corner,
        bool(data["intrafocal"].item())])
    images.append(data["image"].numpy())
    zernikes.append(data["zernikes"])
    dofs.append(data["dof"])

KeyboardInterrupt: 

In [121]:
# convert the metadata into a dataframe
metaDf = pd.DataFrame(
    meta, 
    columns=["pntId", "obsId", "objId", "fx", "fy", "corner", "intra"],
)

# loop through every corner for every pointing
# and randomly select intra/extra pairs
pairs = []
for (pntId, corner), group in metaDf.groupby(["pntId", "corner"]):
    intra = group[group["intra"] == True]
    extra = group[group["intra"] == False]
    idx_pairs = list(zip(intra.index, extra.index))
    pairs.extend(idx_pairs)

In [122]:
# check that everything worked...
for pair in pairs[:5]:
    print(metaDf.iloc[list(pair)], "\n")

      pntId   obsId                objId        fx        fy corner  intra
1570  16305  517986  4634052941090988544  0.020728  0.021682    R44   True
1286  16305  517986  4634051944658577280  0.020444  0.019865    R44  False 

      pntId   obsId                objId        fx        fy corner  intra
1082  18761  927334  3206082005726452480 -0.021923 -0.021728    R00   True
65    18761  927334  3206086262037761024 -0.021598 -0.018981    R00  False 

      pntId   obsId                objId        fx        fy corner  intra
1062  18761  927334  3252337566555130496  0.021507  0.022381    R44   True
1050  18761  927334  3252356911087826432  0.020460  0.020176    R44  False 

      pntId    obsId                objId        fx        fy corner  intra
1071  23605  1972276  5268152120147766784 -0.019549 -0.021278    R00   True
1678  23605  1972276  5268201288933369344 -0.020573 -0.019363    R00  False 

     pntId    obsId                objId        fx        fy corner  intra
377  24225  16